# Python Learning Quest — exercises.ipynb
**Single-file, gamified exercise notebook (Beginner → Advanced)**

Instructions:
- Run the first 6 code cells once (Design System, Persistence, Game Engine, UI helper).
- Re-run the Dashboard (Cell 7) and Navigator (Cell 8) any time to refresh.
- Do exercises in order. Each exercise has: Prompt → Starter → Tests → (optional) Solution.
- Progress is stored in browser `localStorage` when available and can be exported/imported in Cell 4.

**Date:** 2026-01-21

In [ ]:
# Cell 2 — Design tokens (colors, fonts, spacing)
COLORS = {
    'primary_blue': '#00D9FF',
    'accent_pink': '#FF006E',
    'success_green': '#00FF88',
    'warning_gold': '#FFB82C',
    'dark_bg': '#1a1a2e',
    'light_bg': '#0f3460',
    'card_bg': '#16213e',
    'text_primary': '#ffffff',
    'text_secondary': '#b0b0b0',
    'border': '#00D9FF',
    'locked': '#666666',
}
FONTS = {'monospace': 'ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, Courier New, monospace'}
SPACING = {'xs': '4px', 'sm': '8px', 'md': '16px', 'lg': '24px'}
print('Design tokens loaded.')

In [ ]:
# Cell 3 — Persistence helpers: save/load to browser localStorage + export/import
from IPython.display import display, Javascript, HTML
import json
import uuid

DEFAULT_STATE = {
    'player': {
        'id': str(uuid.uuid4()),
        'name': 'Learner',
        'level': 1,
        'xp': 0,
        'xp_to_next': 100,
        'total_xp': 0,
        'streak_current': 0,
        'streak_longest': 0,
        'total_exercises_completed': 0,
        'total_time_spent_minutes': 0,
    },
    'exercises': {},
    'achievements': {},
    'settings': {},
}

LOCAL_STORAGE_KEY = 'python_learning_quest_state'

def save_state_to_browser(state):
    # Best-effort localStorage save. Some hosted environments may block it.
    payload = json.dumps(state)
    js = (
        f"try{{localStorage.setItem('{LOCAL_STORAGE_KEY}', {json.dumps(payload)});}}"
        f"catch(e){{console.warn('localStorage blocked', e);}}"
    )
    display(Javascript(js))

def try_load_state_from_browser():
    # NOTE: Jupyter execution environments differ; this is best-effort.
    # We rely on export/import UI below for guaranteed portability.
    display(Javascript(
        f"""
        (function(){{
          try{{
            const raw = localStorage.getItem('{LOCAL_STORAGE_KEY}');
            if(!raw){{ console.log('No saved state in localStorage'); return; }}
            console.log('State present in localStorage (use Export/Import cell to manage).');
          }} catch(e){{ console.warn('localStorage blocked', e); }}
        }})();
        """
    ))
    return None

def export_state_blob(state):
    # Always works: triggers a JSON download in the browser
    payload = json.dumps(state, indent=2)
    html = f"""
    <script>
      (function(){{
        const dataStr = 'data:application/json;charset=utf-8,' + encodeURIComponent({json.dumps(payload)});
        const a = document.createElement('a');
        a.setAttribute('href', dataStr);
        a.setAttribute('download', 'python_learning_quest_state.json');
        document.body.appendChild(a);
        a.click();
        a.remove();
      }})();
    </script>
    """
    display(HTML(html))

print('Persistence helpers loaded.')

In [ ]:
# Cell 4 — Export / Import UI (browser localStorage)
from IPython.display import HTML, display

html = f"""
<div style="background:{COLORS['light_bg']};padding:12px;border-radius:10px;color:white;max-width:980px;border:1px solid {COLORS['border']}">
  <div style="display:flex;justify-content:space-between;align-items:center">
    <strong style="color:{COLORS['primary_blue']}">Progress Export / Import</strong>
    <span style="font-size:12px;color:{COLORS['text_secondary']}">Storage: browser localStorage (best-effort) + JSON download</span>
  </div>
  <div style="margin-top:10px;display:flex;gap:10px;flex-wrap:wrap;align-items:center">
    <button
      style="padding:8px 12px;border-radius:8px;border:none;cursor:pointer;background:{COLORS['primary_blue']};color:black;font-weight:700"
      onclick="(function(){{
         try{{
           const s = localStorage.getItem('{LOCAL_STORAGE_KEY}');
           if(!s){{ alert('No saved progress in localStorage'); return; }}
           const blob = new Blob([s], {{type: 'application/json'}});
           const url = URL.createObjectURL(blob);
           const a = document.createElement('a'); a.href=url; a.download = 'python_learning_quest_state.json'; a.click();
           URL.revokeObjectURL(url);
         }} catch(e){{ alert('localStorage not available in this environment. Use the Python export_state_blob(STATE).'); }}
      }})()"
    >Export progress (download)</button>

    <input type="file" id="import_state_file" style="color:white">

    <button
      style="padding:8px 12px;border-radius:8px;border:none;cursor:pointer;background:{COLORS['accent_pink']};color:white;font-weight:700"
      onclick="(function(){{
         const f = document.getElementById('import_state_file').files[0];
         if(!f){{ alert('Choose a file first'); return; }}
         const r = new FileReader();
         r.onload = function(e){{
           try{{ localStorage.setItem('{LOCAL_STORAGE_KEY}', e.target.result); }}
           catch(err){{ alert('localStorage blocked. You can still paste JSON into a Python cell and assign STATE manually.'); return; }}
           alert('Imported into browser localStorage. Re-run the Game Engine cell (Cell 5) to reload into STATE.');
         }};
         r.readAsText(f);
      }})()"
    >Import progress (upload)</button>

    <button
      style="padding:8px 12px;border-radius:8px;border:1px solid {COLORS['border']};cursor:pointer;background:transparent;color:white"
      onclick="(function(){{ try{{ localStorage.removeItem('{LOCAL_STORAGE_KEY}'); alert('localStorage progress cleared.'); }} catch(e){{ alert('localStorage blocked.'); }} }})()"
    >Reset local progress</button>
  </div>
  <div style="margin-top:8px;font-size:12px;color:{COLORS['text_secondary']}">
    Tip: If this environment blocks localStorage, use `export_state_blob(STATE)` from a Python cell to download JSON anyway.
  </div>
</div>
"""
display(HTML(html))

In [ ]:
# Cell 5 — Game engine (pure Python)
import datetime

STATE = json.loads(json.dumps(DEFAULT_STATE))  # deep copy

def xp_for_level(level: int) -> int:
    # progressive XP: base 100, +20% per level
    return int(100 * (1.2 ** (level - 1)))

def _recompute_xp_to_next() -> None:
    p = STATE['player']
    p['xp_to_next'] = max(0, xp_for_level(p['level']) - p['xp'])

def award_xp(amount: int, reason: str | None = None) -> dict:
    p = STATE['player']
    p['xp'] += int(amount)
    p['total_xp'] += int(amount)

    # level up loop
    while p['xp'] >= xp_for_level(p['level']):
        p['xp'] -= xp_for_level(p['level'])
        p['level'] += 1

    _recompute_xp_to_next()
    save_state_to_browser(STATE)
    return p

def complete_exercise(key: str, xp_reward: int = 25, minutes_spent: int = 7) -> dict:
    ex = STATE['exercises'].setdefault(key, {})
    ex['status'] = 'completed'
    ex['xp'] = int(xp_reward)
    ex['last_done'] = datetime.date.today().isoformat()

    p = STATE['player']
    p['total_exercises_completed'] += 1
    p['total_time_spent_minutes'] += int(minutes_spent)

    # streak logic (calendar-day based)
    today = datetime.date.today()
    yesterday = (today - datetime.timedelta(days=1)).isoformat()
    last_done = STATE.get('_last_exercise_date')

    if last_done == today.isoformat():
        # already counted today
        pass
    elif last_done == yesterday:
        p['streak_current'] += 1
    else:
        p['streak_current'] = 1

    p['streak_longest'] = max(p['streak_longest'], p['streak_current'])
    STATE['_last_exercise_date'] = today.isoformat()

    award_xp(int(xp_reward), reason=f'Completed {key}')
    save_state_to_browser(STATE)
    return ex

def unlock_achievement(key: str, title: str) -> None:
    ach = STATE['achievements'].setdefault(key, {})
    if ach.get('unlocked'):
        return
    ach['unlocked'] = True
    ach['title'] = title
    ach['date'] = datetime.date.today().isoformat()
    save_state_to_browser(STATE)

def check_achievements() -> None:
    p = STATE['player']
    if p['total_exercises_completed'] >= 1:
        unlock_achievement('first_steps', 'First Steps — Complete 1 exercise')
    if p['total_exercises_completed'] >= 10:
        unlock_achievement('ten_done', 'Getting Serious — Complete 10 exercises')
    if p['streak_current'] >= 7:
        unlock_achievement('week_streak', 'Consistent — 7-day streak')

_recompute_xp_to_next()
print('Game engine ready. Tip: run complete_exercise("1.1", xp_reward=10).')

In [ ]:
# Cell 6 — UI helper (CSS + HTML wrapper)
from IPython.display import HTML, display

def wrap_html(content: str) -> HTML:
    css = f"""
    <style>
      .plq-body {{ font-family: {FONTS['monospace']}; color: {COLORS['text_primary']}; background: linear-gradient(135deg,{COLORS['dark_bg']}, {COLORS['light_bg']}); padding:12px; border-radius:12px; border:1px solid {COLORS['border']}; }}
      .plq-card {{ background: {COLORS['card_bg']}; border:1px solid {COLORS['border']}; padding:12px; border-radius:12px; margin:10px 0; box-shadow: 0 8px 30px rgba(0,0,0,.25); }}
      .plq-row {{ display:flex; gap:12px; flex-wrap:wrap; }}
      .plq-stat {{ flex:1; min-width: 180px; background: rgba(0,0,0,.25); border:1px solid rgba(255,255,255,.08); padding:10px; border-radius:10px; }}
      .plq-muted {{ color: {COLORS['text_secondary']}; }}
      .plq-btn {{ padding:8px 12px; border-radius:10px; border:none; cursor:pointer; font-weight:700; }}
      .plq-btn-primary {{ background:{COLORS['primary_blue']}; color:#000; }}
      .plq-btn-secondary {{ background:{COLORS['accent_pink']}; color:#fff; }}
      .plq-btn-ghost {{ background:transparent; border:1px solid {COLORS['border']}; color:#fff; }}
      .plq-progress {{ height:16px; background: rgba(0,0,0,.35); border-radius:999px; overflow:hidden; border:1px solid rgba(255,255,255,.08); }}
      .plq-progress > div {{ height:100%; background: linear-gradient(90deg,{COLORS['primary_blue']},{COLORS['accent_pink']}); }}
      code {{ background: rgba(0,0,0,.35); padding: 2px 6px; border-radius: 6px; }}
      a {{ color: {COLORS['primary_blue']}; text-decoration: none; }}
      a:hover {{ text-decoration: underline; }}
    </style>
    """
    return HTML(css + f"<div class='plq-body'>{content}</div>")

print('UI helper ready.')

In [ ]:
# Cell 7 — Dashboard
from IPython.display import display

def render_dashboard() -> None:
    check_achievements()
    p = STATE['player']
    denom = max(1, xp_for_level(p['level']))
    progress_pct = int((p['xp'] / denom) * 100)

    recent_ach = [a for a in STATE.get('achievements', {}).values() if a.get('unlocked')]
    recent_ach = sorted(recent_ach, key=lambda x: x.get('date', ''), reverse=True)[:3]
    ach_html = ''.join([f"<div class='plq-muted'>🏅 {a.get('title','(badge)')} <span class='plq-muted'>({a.get('date','')})</span></div>" for a in recent_ach])
    if not ach_html:
        ach_html = "<div class='plq-muted'>No achievements yet — complete an exercise to unlock!</div>"

    html = f"""
    <div class='plq-card'>
      <div style='display:flex;justify-content:space-between;align-items:flex-end;gap:12px;flex-wrap:wrap'>
        <div>
          <div style='font-size:20px;font-weight:900;color:{COLORS['primary_blue']}'>PYTHON QUEST — Dashboard</div>
          <div class='plq-muted'>Single-file progress + XP system</div>
        </div>
        <div class='plq-muted'>Player: <strong>{p['name']}</strong> • ID: {p['id'][:8]}</div>
      </div>

      <div class='plq-row' style='margin-top:12px'>
        <div class='plq-stat'>
          <div>Level: <strong>{p['level']}</strong></div>
          <div>XP: <strong>{p['xp']}</strong> / {xp_for_level(p['level'])}</div>
          <div class='plq-progress' style='margin-top:8px'><div style='width:{progress_pct}%;'></div></div>
          <div class='plq-muted' style='margin-top:8px'>To next: {p['xp_to_next']} XP</div>
        </div>
        <div class='plq-stat'>
          <div>Total XP: <strong>{p['total_xp']}</strong></div>
          <div>Exercises done: <strong>{p['total_exercises_completed']}</strong></div>
          <div>Time spent: <strong>{p['total_time_spent_minutes']}</strong> min</div>
        </div>
        <div class='plq-stat'>
          <div>Streak: <strong>{p['streak_current']}</strong></div>
          <div>Longest: <strong>{p['streak_longest']}</strong></div>
          <div style='margin-top:8px'>{ach_html}</div>
        </div>
      </div>

      <div style='margin-top:12px;display:flex;gap:10px;flex-wrap:wrap'>
        <button class='plq-btn plq-btn-primary' onclick="alert('Run in Python: complete_exercise(\"demo.topic1_ex1\", xp_reward=25, minutes_spent=7) then re-run the Dashboard (Cell 7).')">Demo: how to mark complete</button>
        <button class='plq-btn plq-btn-ghost' onclick="alert('Tip: use export/import in Cell 4 to move progress between devices.')">Help</button>
      </div>
    </div>
    """
    display(wrap_html(html))

render_dashboard()

In [ ]:
# Cell 8 — Navigator (topics + quick reference)
from IPython.display import display

TOPICS = [
    ('1. Intro & Output', ['1.1 Hello World', '1.2 Multi-line output', '1.3 Interactive bio']),
    ('2. Variables & Data Types', ['2.1 Assignments', '2.2 Type conversion', '2.3 Flexible calculator']),
    ('6. Lists', ['6.1 List operations', '6.2 List comprehensions', '6.3 Moving average']),
    ('15. Classes & OOP', ['15.1 Basic Class', '15.2 Inheritance', '15.3 Vector engine (mini)']),
]

def render_nav() -> None:
    rows = ''
    for phase, items in TOPICS:
        items_html = '<br>'.join([f"<span class='plq-muted'>•</span> {itm}" for itm in items])
        rows += (
            f"<div class='plq-card'>"
            f"<div style='font-weight:900;color:{COLORS['primary_blue']}'>{phase}</div>"
            f"<div class='plq-muted' style='margin-top:8px;line-height:1.6'>{items_html}</div>"
            f"</div>"
        )
    display(wrap_html(f"<div style='max-width:980px'>{rows}</div>"))

render_nav()

In [ ]:
# Cell 9 — Reusable Exercise Card renderer (pure notebook)
from IPython.display import HTML, display
import html as _html

class ExerciseCardRenderer:
    def __init__(self, id_: str, title: str, difficulty: str, xp: int):
        self.id = id_
        self.title = title
        self.difficulty = difficulty
        self.xp = int(xp)

    def render(self) -> None:
        safe_title = _html.escape(self.title)
        status = STATE.get('exercises', {}).get(self.id, {}).get('status', 'available')
        badge = '✅ Completed' if status == 'completed' else '🟦 Available'

        html_code = f"""
        <div class='plq-card' style='max-width:980px'>
          <div style='display:flex;justify-content:space-between;gap:12px;flex-wrap:wrap;align-items:center'>
            <div style='font-weight:900;color:{COLORS['primary_blue']}'>{self.id} — {safe_title}</div>
            <div class='plq-muted'>Difficulty: {self.difficulty} • {self.xp} XP • {badge}</div>
          </div>
          <div style='margin-top:10px;display:flex;gap:10px;flex-wrap:wrap'>
            <button class='plq-btn plq-btn-primary' onclick="alert('Run in Python: complete_exercise(\"{self.id}\", xp_reward={self.xp}, minutes_spent=7); then re-run the Dashboard (Cell 7).')">I finished — mark complete</button>
            <button class='plq-btn plq-btn-ghost' onclick="alert('Hint: read the exercise prompt and the starter docstring carefully. Write a simple version first, then add edge cases.')">Hint</button>
          </div>
        </div>
        """
        display(HTML(html_code))

print('ExerciseCardRenderer ready. Example: ExerciseCardRenderer("1.1", "Hello World", "Basic", 10).render()')

## Exercise 1.1 — Hello World (Basic)
**Challenge**
- Return the exact string `"Hello, World!"`.
- Add a single-line comment and a multi-line comment (in your code cell).

**Expected output**
```
Hello, World!
```

In [ ]:
# Exercise 1.1 starter
ExerciseCardRenderer('1.1', 'Hello World', 'Basic', 10).render()

def exercise_1_1() -> str:
    """Return the string that should be printed."""
    # TODO: implement
    raise NotImplementedError

# Quick self-test (uncomment after implementing)
# assert exercise_1_1() == 'Hello, World!'
# print('Exercise 1.1 OK')

## Exercise 2.2 — Type Conversion (Intermediate)
Implement `convert_inputs(age_text, height_text, subscribed_text)` and return:
- `age` as `int`
- `height` as `float`
- `subscribed` as `bool` (accept yes/no, true/false, y/n, 1/0; case-insensitive)

Raise `ValueError` for invalid inputs.

In [ ]:
ExerciseCardRenderer('2.2', 'Type Conversion', 'Intermediate', 25).render()

def convert_inputs(age_text: str, height_text: str, subscribed_text: str) -> dict[str, object]:
    """Convert user text inputs into typed values with validation.

    Returns: {'age': int, 'height': float, 'subscribed': bool}
    """
    # TODO: implement
    raise NotImplementedError

def _t_2_2():
    assert convert_inputs('21', '1.75', 'YES') == {'age': 21, 'height': 1.75, 'subscribed': True}
    assert convert_inputs('21', '1.75', 'no') == {'age': 21, 'height': 1.75, 'subscribed': False}

# run_tests_and_award('2.2', _t_2_2, xp_reward=25)

## Exercise 6.3 — Moving Average (Advanced)
Implement a sliding-window moving average.

Rules:
- If `window <= 0`, raise `ValueError`.
- If `window > len(values)`, return `[]`.

Example: `moving_average([1,2,3,4], 2) -> [1.5, 2.5, 3.5]`

In [ ]:
ExerciseCardRenderer('6.3', 'Moving Average', 'Advanced', 50).render()

def moving_average(values: list[float], window: int) -> list[float]:
    """Return moving average using a sliding window."""
    # TODO: implement
    raise NotImplementedError

def _t_6_3():
    assert moving_average([1, 2, 3, 4], 2) == [1.5, 2.5, 3.5]
    assert moving_average([10, 20, 30], 3) == [20.0]
    assert moving_average([1, 2], 3) == []

# run_tests_and_award('6.3', _t_6_3, xp_reward=50)

## Test Harness (inline grading)
Use this to run tests and automatically award XP on success.

In [ ]:
# Cell — Simple test harness
def run_tests_and_award(exercise_id: str, test_func, xp_reward: int = 25):
    try:
        test_func()
        complete_exercise(exercise_id, xp_reward=xp_reward, minutes_spent=7)
        print(f'✅ All tests passed. Awarded {xp_reward} XP for {exercise_id}.')
    except AssertionError as e:
        print('❌ Tests failed:', e)
    except Exception as e:
        print('❌ Error running tests:', e)

print('Test harness ready. Example: run_tests_and_award("1.1", _t_1_1, xp_reward=10)')

## Analytics (progress chart)
Optional visualization of your current level progress.

In [ ]:
import matplotlib.pyplot as plt

def plot_progress():
    p = STATE['player']
    xp_needed = max(0, xp_for_level(p['level']) - p['xp'])
    labels = ['XP gained', 'XP remaining']
    sizes = [p['xp'], xp_needed]
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.pie(sizes, labels=labels, autopct='%1.1f%%')
    ax.set_title(f'Level {p["level"]} progress')
    plt.show()

plot_progress()